In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [4]:
routine_full = pd.read_csv('../../data/02_intermediate/routine_inter.csv')

/Users/Kate/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
routine_full.head(2)

,Sample,Prod,DateSampled,Grade,DateTested,HSID,Supplier,Gallons,FacilityName,SiteAddress,Test,Units,Method,Result,MinResult,MaxResult,Compliance
0,61916134,Gasoline,11/23/2015 12:00:00 AM,Mid Grade Unleaded,NaN,15752.0,Marathon Petroleum Lp,2498.6,Circle K #2723609,"198 Haywood Ln \r\nnashville, Tn 37211",Antiknock Index,,R+M/2,89.8,87,,Y
1,61916134,Gasoline,11/23/2015 12:00:00 AM,Mid Grade Unleaded,NaN,15752.0,Marathon Petroleum Lp,2498.6,Circle K #2723609,"198 Haywood Ln \r\nnashville, Tn 37211",API Gravity,,D4052,60.9,,,


In [38]:
routine_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1129178 entries, 0 to 1129218
Data columns (total 14 columns):
Sample          1129178 non-null object
Prod            1129178 non-null object
DateSampled     1129178 non-null datetime64[ns]
Grade           1129178 non-null object
Supplier        949928 non-null object
FacilityName    1129178 non-null object
SiteAddress     1121938 non-null object
Test            1124013 non-null object
Units           1123779 non-null object
Method          1123779 non-null object
Result          1068687 non-null object
MinResult       994234 non-null object
MaxResult       1008914 non-null object
Compliance      1129178 non-null object
dtypes: datetime64[ns](1), object(13)
memory usage: 129.2+ MB


#### Sample Variable

This is a unique identifier for each sample taken. As such, it should be a string (as it is according to info). 

#### Prod

There are two missing product entries. Since this is 2 entries in over 1million, I will drop the two missing entries

In [7]:
routine_full.dropna(subset=['Prod'], inplace=True)

#### DateSampled

This is entered as a float in the original dataset, but should be datetime

In [8]:
routine_full['DateSampled'] = pd.to_datetime(routine_full['DateSampled'])

The expert we intereviewed mentioned that the time portion of the datetime column is innacurate. Because it doesn't add information I will remove it from the dataframe. 

In [9]:
routine_full['DateSampled'] = pd.DatetimeIndex(routine_full['DateSampled']).normalize()

#### Grade

This variable does not need to be updated right now. 

#### DateTested

The expert we interviewed stated that teh datetested column is a relic from the old system. Most of the Datetested column is empty. Because of this we will drop it from our analysis

In [10]:
routine_full.drop(['DateTested'], axis=1, inplace=True)

#### HSID

Our expert suggested dropping this from the sample. He believes it is a relic from the old system

In [11]:
routine_full.drop(['HSID'], axis=1, inplace=True)

#### <font color='red'>Supplier</font>

~15% of the supplier column for the entire dataset is empty. 

In [12]:
routine_full['Supplier'].isna().sum()/len(routine_full)

0.15877284879699827

Since we are interested in Gasoline and Diesel let's check out the supplier na count for these two products seperatly

~15% of gasoline supplier column is N/A

In [13]:
gas = routine_full.loc[routine_full['Prod'] == 'Gasoline']
gas['Supplier'].isna().sum()/len(gas)

0.1556561131297752

In [14]:
gas[gas['Supplier'].isna()]

,Sample,Prod,DateSampled,Grade,Supplier,Gallons,FacilityName,SiteAddress,Test,Units,Method,Result,MinResult,MaxResult,Compliance
39,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Antiknock Index,,R+M/2,87.8,87,,Y
40,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",API Gravity,,D4052,63.7,,,
41,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",DIPE,Vol. %,D4815,ND,,,
42,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 10%,Deg. C,D86,43.5,,,Y
43,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 20%,Deg. C,D86,50.0,,,
44,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 30%,Deg. C,D86,56.8,,,
45,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 40%,Deg. C,D86,62.4,,,
46,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 5%,Deg. C,D86,39.6,,,
47,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 50%,Deg. C,D86,67.4,,,Y
48,61916135,Gasoline,2015-11-24,Regular Unleaded,NaN,8874,Tiger Market #214,"2001 8th Ave S \r\nnashville, Tn 37204",Distillation 60%,Deg. C,D86,95.7,,,


~ around 6% of diesel supplier column is N/A

In [15]:
diesel = routine_full.loc[routine_full['Prod'] == 'Diesel']
diesel['Supplier'].isna().sum()/len(gas)

0.062343330937647906

#### Gallons

Known Problems
1. Contains a number of Nan Values
1. Many values contain inches (relic from old system). These need to be taken out

Later we might come back and deal with this dataset if we can find a good use for it. But for now we will drop it. 

In [16]:
# gallons_test = routine_full

In [17]:
# gallons_test.dropna(subset=['Gallons'], inplace=True)

In [18]:
# gallons_test.info()

There are a number of inch measurements in the gallons column. Since these are not useful, we will identify the inch variables and conver them to nan.

In [19]:
# gallons_full = routine_full.copy()

In [20]:
# gallons_full.loc[gallons_full['Gallons'].str.contains('"')==True] = np.nan

In [21]:
# gallons_full.drop(['Sample', 'Prod', 'DateSampled', 'Grade', 'Supplier',
#        'FacilityName', 'SiteAddress', 'Test', 'Units', 'Method', 'Result',
#        'MinResult', 'MaxResult', 'Compliance'], axis=1, inplace=True)

In [22]:
# routine_full['Gallons'] = gallons_full

In [23]:
# routine_full['Gallons'] = routine_full['Gallons'].str.replace(',', '')

In [24]:
routine_full.drop(['Gallons'], axis=1, inplace=True)

#### FacilityName

In [25]:
routine_full['FacilityName'].isna().sum()

39

There are only 39 rows without a facility name. Since the number is low I will drop the rows without facility 

In [26]:
routine_full.dropna(subset=['FacilityName'], inplace=True)

#### SiteAddress

In [27]:
routine_full.loc[routine_full['FacilityName']=="FILL 'N' SAVE #3"]

,Sample,Prod,DateSampled,Grade,Supplier,FacilityName,SiteAddress,Test,Units,Method,Result,MinResult,MaxResult,Compliance
171707,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Antiknock Index,,R+M/2,87.2,87,NaN,Y
171708,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,API Gravity,,D4052,58.9,NaN,NaN,
171709,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,DIPE,Vol. %,D4815,ND,NaN,NaN,
171710,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 10%,Deg. C,D86,50.2,NaN,60,Y
171711,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 20%,Deg. C,D86,56.3,NaN,NaN,
171712,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 30%,Deg. C,D86,61.7,NaN,NaN,
171713,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 40%,Deg. C,D86,66.3,NaN,NaN,
171714,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 5%,Deg. C,D86,46.3,NaN,NaN,
171715,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 50%,Deg. C,D86,85.7,66,116,Y
171716,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 60%,Deg. C,D86,107.7,NaN,NaN,


Ask John why there are missing addresses

In [28]:
routine_full[routine_full['SiteAddress'].isna()==True]

,Sample,Prod,DateSampled,Grade,Supplier,FacilityName,SiteAddress,Test,Units,Method,Result,MinResult,MaxResult,Compliance
171707,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Antiknock Index,,R+M/2,87.2,87,NaN,Y
171708,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,API Gravity,,D4052,58.9,NaN,NaN,
171709,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,DIPE,Vol. %,D4815,ND,NaN,NaN,
171710,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 10%,Deg. C,D86,50.2,NaN,60,Y
171711,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 20%,Deg. C,D86,56.3,NaN,NaN,
171712,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 30%,Deg. C,D86,61.7,NaN,NaN,
171713,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 40%,Deg. C,D86,66.3,NaN,NaN,
171714,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 5%,Deg. C,D86,46.3,NaN,NaN,
171715,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 50%,Deg. C,D86,85.7,66,116,Y
171716,ADUN-A9SJQ3,Gasoline,2016-05-09,Regular Unleaded,NFRY-A9M4A5,FILL 'N' SAVE #3,NaN,Distillation 60%,Deg. C,D86,107.7,NaN,NaN,


#### Complaince

Test contains 5 unique values, 'Y', ' ', 'N', nan, and 'Select'. From speaking with our expert we know that ' ', nan and select are all the same (none values. Not all tests are for compliance purposes). So we will make a seperate catagory called None. 

In [29]:
routine_full['Test'].isna().sum()

5165

In [30]:
routine_full.Compliance.unique()

array(['Y', ' ', 'N', nan, 'Select'], dtype=object)

In [31]:
routine_full['Compliance'] = routine_full['Compliance'].str.replace(' ', 'None')

In [32]:
routine_full.Compliance.unique()

array(['Y', 'None', 'N', nan, 'Select'], dtype=object)

In [33]:
routine_full['Compliance'] = routine_full['Compliance'].str.replace('Select', 'None')

In [34]:
routine_full.Compliance.unique()

array(['Y', 'None', 'N', nan], dtype=object)

In [33]:
routine_full['Compliance'] = routine_full['Compliance'].replace(np.nan, 'None')

In [34]:
routine_full.Compliance.unique()

array(['Y', 'None', 'N'], dtype=object)

Let's reduce the tests we are working with to Distillation 50%, Vapor Pressure and Vapor-Liquid Ratio

In [35]:
# routine_test_key = routine_full.loc[(routine_full['Test']=='Distillation 50%') 
#                  | (routine_full['Test']=='Vapor Pressure') 
#                  | (routine_full['Test']=='Vapor-Liquid Ratio')]

In [37]:
# routine_test_key.to_csv('../../data/02_intermediate/routine_clean.csv', index=False)
routine_full.to_csv('../../data/02_intermediate/routine_clean.csv', index=False)